# Details about the Max-Cut for a random 72-vertex graph

`G72.txt` contains the adjacency matrix of the graph.   
`G72PhiA.txt` contains the output $\Phi_A=\mathrm{Tr}_B(\Phi_{AB})$ of the SDP.

The main difficulty in this example is that the cut achieving the Max-Cut is not unique.  
This results in a mixed output state $\Phi_A$ and hence we need to guess the cuts.

In [1]:
import numpy as np
#import networkx as nx
from itertools import product

In [2]:
W = np.loadtxt("G72.txt")
PhiA = np.loadtxt("G72PhiA.txt")

In [3]:
print("Number of vertices:", W.shape[0])
print("Number of edges:", np.sum(W)/2)

#print("The Adjacency Matrix:\n", W)
#print("The graph6 format:\n", nx.to_graph6_bytes(nx.from_numpy_array(W)))

Number of vertices: 72
Number of edges: 475.0


In [4]:
# The final result of the SDP
# (output of the SCS solver, up to a relative precision ~1e-5)
# This gives an upper bound of the Max-Cut $\xi_2=321$

print("Final result of the SDP:", -np.trace(W@PhiA)/4+np.sum(W)/4)

Final result of the SDP: 320.99748309970926


In [5]:
# Take the first row of PhiA and guess the cut

mu = PhiA[0]
print("The first row of PhiA:\n", mu)

The first row of PhiA:
 [ 0.99999772  0.1366566   0.40239862 -0.40240242  0.08143387  0.13665396
  0.69766212 -0.25195801  0.13665601  0.99996416  0.13665593 -0.99996592
 -0.99996608 -0.13665671  0.99996484  0.13665536 -0.13665611 -0.13665588
  0.13665591 -0.13665592  0.13665556  0.47971125 -0.13665668 -0.4796908
  0.13665657  0.13665589  0.13665561  0.1366563   0.13665545 -0.7641326
  0.99996554 -0.13665642 -0.99996514 -0.64055099 -0.13665638 -0.99996478
 -0.13665622  0.9999646   0.13665615 -0.27424759  0.50094135 -0.99996472
  0.24585829  0.12577103 -0.13665642  0.13665588  0.76414238 -0.13665639
  0.13665627  0.13665618  0.13665464 -0.13665642 -0.1366563  -0.13665602
  0.13665515 -0.13665634 -0.1366562   0.13665489  0.13665524 -0.50094243
 -0.45743093  0.23934034 -0.13665632 -0.99996514 -0.1366557   0.99996378
  0.13665617  0.13665593 -0.13665687 -0.6191361   0.32737854  0.99996413]


In [6]:
# Show that $\xi_2=321$ is an achievalbe upper bound
print("The upper bound 321 is achieved by the following cuts:\n")

# number of optimal cuts
num = 0


# initialize the guess
y = np.zeros(72)

# set the entries to one if they are close to
y[mu > 0.9999] = 1
y[mu < -0.9999] = -1

# guess some entries are equal if they are close to each other (one possibility)
y[abs(mu - 0.1366) < 0.0001] = 1
y[abs(mu + 0.1366) < 0.0001] = -1

# run an iteration on the undetermined entries
undet = np.where(np.isclose(y, 0))
for i in product([1, -1], repeat=np.shape(undet)[1]):
    y[undet] = i
    if np.isclose(-y@W@y/4+np.sum(W)/4, 321):
        print("x =", (y).astype(int), "\n")
        #print(np.where(y>0))
        #print("cut_size =", -y@W@y/4+np.sum(W)/4)
        num += 1
    if -y@W@y/4+np.sum(W)/4 > 321.5:
        print("Error!")
        
        
# initialize the guess
z = np.zeros(72)

# set the entries to one if they are close to
z[mu > 0.9999] = 1
z[mu < -0.9999] = -1       

# guess some entries are equal if they are close to each other (the other possibility)
z[abs(mu - 0.1366) < 0.0001] = -1
z[abs(mu + 0.1366) < 0.0001] = 1
# run an iteration on the undetermined entries
undet = np.where(np.isclose(z, 0))
for i in product([1, -1], repeat=np.shape(undet)[1]):
    z[undet] = i
    if np.isclose(-z@W@z/4+np.sum(W)/4, 321):
        print("x =", (z).astype(int), "\n")
        #print(np.where(z>0))
        #print("cut_size =", -z@W@z/4+np.sum(W)/4)
        num += 1
    if -z@W@z/4+np.sum(W)/4 > 321.5:
        print("Error!")
        
print("Number of optimal cuts found:", num)

The upper bound 321 is achieved by the following cuts:

x = [ 1  1  1 -1 -1  1  1 -1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1  1 -1 -1  1
  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1  1 -1  1  1 -1
  1  1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1  1 -1 -1 -1  1  1  1 -1 -1  1  1] 

x = [ 1  1  1 -1 -1  1 -1  1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1  1  1 -1 -1
  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1 -1  1  1  1  1 -1  1 -1 -1  1  1 -1
  1  1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1  1 -1 -1 -1  1  1  1 -1 -1 -1  1] 

x = [ 1  1 -1  1  1  1  1 -1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1  1  1 -1 -1
  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1 -1 -1  1  1 -1
  1  1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1 -1 -1 -1 -1  1  1  1 -1  1  1  1] 

x = [ 1  1 -1  1 -1  1  1  1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1  1  1 -1 -1
  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1  1 -1 -1  1  1 -1
  1  1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1  1 -1 -1 -1  1  1  1 -1 -1  1  1] 

x = [ 1  1 -1  1 -1  1  1  1  1  1  1 -1